In [12]:
import json
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import accuracy_score
import torch

class TextClassifier:
    def __init__(self, model_name="Qwen/Qwen3-0.6B"):
        # Уменьшаем потребление памяти
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
            low_cpu_mem_usage=True
        )
        # Освобождаем память
        torch.cuda.empty_cache()

        self.classes = ["sport", "technology", "culinary", "politics", "health"]
        self.temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]
        self.examples = self._create_examples()

    def _create_examples(self):
        return [
            {"text": "Баскетболисты ЦСКА одержали уверенную победу над испанским «Реалом» в Евролиге.", "label": "sport"},
            {"text": "Квантовые компьютеры MIT научились решать задачи за секунды, которые обычным системам требовались дни.", "label": "technology"},
            {"text": "Как приготовить домашнюю лапшу рамен за 30 минут — пошаговый гид с фотографиями.", "label": "culinary"},
            {"text": "Выборы в Европарламент 2024: рост популярности экологических партий в Германии и Франции.", "label": "politics"},
            {"text": "Психологические практики: эффективные методы борьбы с тревожностью без лекарств.", "label": "health"}
        ]

    def _build_prompt(self, text, examples=None):
        system_msg = "Вы классифицируете текст по категориям: sport, technology, culinary, politics, health. Ответьте только названием категории."
        user_msg = f"Текст: {text}\nКатегория:"

        if examples:
            user_msg = ""
            for ex in examples:
                user_msg += f"\nТекст: {ex['text']}\nКатегория: {ex['label']}\n"
            user_msg += f"\nТекст: {text}\nКатегория:"

        return [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg.strip()}
        ]

    def _generate(self, text, examples=None, temperature=0.7, max_tokens=100):
        try:
            messages = self._build_prompt(text, examples)
            prompt = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(self.model.device)

            # Уменьшаем максимальное количество токенов
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

            output_ids = outputs[0][inputs.input_ids.shape[1]:]
            return self.tokenizer.decode(output_ids, skip_special_tokens=True).strip()
        except Exception as e:
            print(f"Ошибка генерации: {str(e)}")
            return ""

    def _parse_output(self, output):
        # Простая проверка без JSON
        output = output.lower().replace('"', '').replace("'", "").strip()
        for cls in self.classes:
            if cls in output:
                return {c: 1 if c == cls else 0 for c in self.classes}
        return {cls: 0 for cls in self.classes}

    def classify_zero_shot(self, text):
        return self._parse_output(self._generate(text))

    def classify_one_shot(self, text):
        return self._parse_output(self._generate(text, examples=[self.examples[0]]))

    def classify_few_shot(self, text):
        return self._parse_output(self._generate(text, examples=self.examples))

    def probabilistic_classify(self, text, iterations=3):
        results = {cls: 0 for cls in self.classes}
        for _ in range(iterations):
            temp = random.choice(self.temperatures)
            pred = self.classify_zero_shot(text)  # Используем zero-shot для скорости
            for cls in self.classes:
                results[cls] += pred[cls]
        total = sum(results.values())
        return {k: v/total if total else 1/len(self.classes) for k, v in results.items()}

    def evaluate(self, texts, true_labels):
        results = {
            "Zero-shot": [],
            "One-shot": [],
            "Few-shot": [],
            "Probabilistic": []
        }

        for i, text in enumerate(texts):
            print(f"\nОбработка текста {i+1}/{len(texts)}")

            # Zero-shot
            zero_pred = self.classify_zero_shot(text)
            results["Zero-shot"].append(max(zero_pred, key=zero_pred.get))

            # One-shot
            one_pred = self.classify_one_shot(text)
            results["One-shot"].append(max(one_pred, key=one_pred.get))

            # Few-shot
            few_pred = self.classify_few_shot(text)
            results["Few-shot"].append(max(few_pred, key=few_pred.get))

            # Probabilistic
            prob_pred = self.probabilistic_classify(text)
            results["Probabilistic"].append(max(prob_pred, key=prob_pred.get))

        # Расчет точности
        scores = {}
        for method, preds in results.items():
            scores[method] = accuracy_score(true_labels, preds)

        return scores

if __name__ == "__main__":
    samples = [
        ("Футбольный матч между сборными России и Германии завершился со счётом 2:1 в пользу хозяев.", "sport"),
        ("Блокчейн-платформа Ethereum перешла на энергоэффективный алгоритм PoS, снизив потребление энергии на 99.95%.", "technology"),
        ("Постный борщ с фасолью и свеклой: вкусный и полезный вариант для вегетарианцев.", "culinary"),
        ("Совет Безопасности ООН провёл экстренное заседание по ситуации на Ближнем Востоке.", "politics"),
        ("Исследование Университета Гарварда: регулярные прогулки снижают риск сердечно-сосудистых заболеваний на 40%.", "health")
    ]

    texts = [item[0] for item in samples]
    labels = [item[1] for item in samples]

    print("Инициализация модели...")
    classifier = TextClassifier()

    print("Запуск оценки...")
    scores = classifier.evaluate(texts, labels)

    print("\nРезультаты:")
    for method, accuracy in scores.items():
        print(f"{method}: {accuracy:.2f}")

Инициализация модели...
Запуск оценки...

Обработка текста 1/5

Обработка текста 2/5

Обработка текста 3/5

Обработка текста 4/5

Обработка текста 5/5

Результаты:
Zero-shot: 0.20
One-shot: 0.20
Few-shot: 0.20
Probabilistic: 0.20


**Выводы:**

* Разработана система классификации текстов на 5 категорий (sport, technology, culinary, politics, health)

Реализованы 4 метода классификации:

Zero-shot (классификация только по инструкции)

One-shot (с одним примером)

Few-shot (с несколькими примерами)

Вероятностный подход (усреднение многократных предсказаний)

Протестирована модель Qwen-0.6B на 5 текстах с известными метками

Рассчитана точность каждого метода

* Текущая реализация показала неудовлетворительные результаты (20% точности), что указывает на необходимость фундаментального пересмотра подхода. Одинаково низкие результаты всех методов классификации свидетельствуют о системной проблеме, вероятно связанной с:

Неадекватностью промптов для данной модели

Ошибками в обработке ответов

Фундаментальным несоответствием между задачей и возможностями модели